# Usage

`liana` contains different statistical methods to infer `ligand-receptor` interactions from single-cell transcriptomics data omics data using prior knowledge.
In this notebook we showcase how to use liana in its' most basic form with toy data.

<div class="alert alert-info">

**Note**
    
In Jupyter notebooks and lab, you can see the documentation for a python function by hitting `SHIFT + TAB`, hit it twice to expand the view, or by typing `?name_of_function`.

</div>  

## Loading Packages

In [ ]:
import liana as li

# Only needed for visualization:
import matplotlib.pyplot as plt
from scanpy.pl import umap

# toy data
from scanpy.datasets import pbmc68k_reduced
%matplotlib inline

## Loading toy data

`liana` uses `anndata` objects with processed single-cell (*and soon spatial*) transcriptomics data, with pre-defined cell labels (identities), to predict ligand-receptor interactions among all pairs of cell identities.

To load the example data-set, simply run:

In [ ]:
adata = pbmc68k_reduced()

The example single-cell data consists processed data with PBMCs cell types

In [ ]:
umap(adata, color='bulk_labels', title='', frameon=False)

## Background

In the most basic scenario, `liana` works with the log-trasformed counts matrix, in this object the normalized counts are stored in `raw`:

In [ ]:
adata.raw.X

Preferably, one would use `liana` with all features (genes) for which we have enough counts, but for the sake of this tutorial we are working with a matrix pre-filtered to the variable features alone.

In the background, `liana` aggregates the counts matrix and generates statistics, typically related to cell identies.
These statistics are then utilized by each of the methods in `liana`.

### Methods

In [ ]:
li.show_methods()

Each method infers relevant ligand-receptor interactions relying on different assumptions, and each returns different ligand-receptor scores, typically a pair per method. One score corresponding to
the `magnitude` (strength) of interaction and the other reflecting how `specificity` of a given interaction to a pair cell identities.

<div class="alert alert-info">

<h4> Method Class</h4>
    
Methods in liana are **callable** instances of the `Method` class. To obtain further information for each method the user can refer to the methods documentation `?method_name` or `?method.__call__`. Alternatively, users can use the `method.describe` function to get a short summary for each method.

</div>  


For example, if the user wishes to learn more about liana's `rank_aggregate` implementation, where we combine the scores of multiple methods, they could do the following: 

In [ ]:
# import liana's rank_aggregate
from liana import cellphonedb

In [ ]:
?cellphonedb.__call__

or alternatively:

In [ ]:
cellphonedb.describe()

#### Resources

To provide ligand-receptor predictions `liana` relies on prior-knowledge and makes use of the following ligand-receptor resources:

In [ ]:
li.show_resources()

These were pre-generated using the [OmniPath](https://github.com/saezlab/omnipath) meta-database, but any custom resource can also be passed.
We refer the user to [OmniPath](https://github.com/saezlab/omnipath) as a useful tool to build any resource.

<div class="alert alert-info">
    
By default, `liana` uses the `consensus` resource, which is composed by multiple expert-curated ligand-receptor resources, including CellPhoneDB, CellChat, ICELLNET, connectomeDB2020, and CellTalkDB.
</div>  


## Example Run

### Individual Methods

In [ ]:
# import indivudla methods
from liana import singlecellsignalr, connectome, cellphonedb, natmi, logfc

In [ ]:
# run cellphonedb
adata = cellphonedb(adata, groupby='bulk_labels', use_raw=True, expr_prop=0.1)

By default, results will be assigned to `adata.uns['liana_res']`

In [ ]:
# by default, liana's output is saved in place:
adata.uns['liana_res'].head()

Here, we see that generic stats are provided for both `ligand` and `receptor` *entities*, more specifically:
- `ligand` and `receptor` are typically the two entities that interact. Though it should be noted that these are not limited to secreted signalling. Also, in the case of heteromeric complexes, these represent the subunit with minimum expression, while `*_complex` corresponds to the actual complex, and is hence the unique key for each row. 
- `source` and `target` columns represent the `source/sender` and `target/receiver` cell identity for each interaction, respectively
- `*_props`: represents the proportion of cells that express the entity
- `*_pvals`: 1vsRest p-value calculated at the single-cell level
- `*_means`: entity expression mean per cell type
- `lr_means`: mean ligand-receptor expression, as a measure of ligand-receptor interaction *magnitude*
- `pvals`: permutation-based p-value, as a measure of interaction *specificity*

Note that `ligand`, `receptor`, `source`, `target` columns are returned by every method, while the rest can vary across the methods.

#### Dotplot

We can now visualize the results that we just obtained.

LIANA provides some basic, but flexible plotting functionalities. Here, we will generate a dotplot of relevant ligand-receptor interactions.

In [ ]:
my_p = li.pl.dotplot(adata, 
                     colour='lr_means',
                     size='pvals',
                     inverse_size=True, # we inverse sign since we want small p-values to have large sizes
                     source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
                     target_labels=['CD34+', 'CD56+ NK'],
                     figure_size=(8, 7),
                     # finally, since cpdbv2 suggests using a filter to FPs
                     # we filter the pvals column to <= 0.05
                     filterby='pvals',
                     filter_lambda=lambda x: x <= 0.05
                    )

In [ ]:
my_p

<div class="alert alert-info">
Missing dots here would represent interactions for which the ligand and receptor are not expressed above the `expr_prop`.
</div>

### Rank Aggregate
In addition to the individual methods, LIANA also provides a consensus that integrates the predictions of individual methods.
This is done by ranking and aggregating ([RRA](https://academic.oup.com/bioinformatics/article-abstract/28/4/573/213339)) the ligand-receptor interaction predictions from all methods.

In [ ]:
# import rank_aggregate methods
from liana import rank_aggregate

In [ ]:
# Run method
adata = rank_aggregate(adata, groupby='bulk_labels', use_raw=True, expr_prop=0)

In [ ]:
adata.uns['liana_res'].head()

In [ ]:
rank_aggregate.describe()

#### Dotplot

We will now plot the most 'relevant' interactions ordered to the `steady_rank` results from aggregated_rank.

In [ ]:
my_p = li.pl.dotplot(adata, 
                     colour='lrscore',
                     size='specificity_rank',
                     inverse_size=True,
                     source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
                     target_labels=['CD34+', 'CD56+ NK'],
                     top_n=20, 
                     orderby='steady_rank',
                     orderby_ascending=True,
                     figure_size=(8, 7)
                    )

In [ ]:
my_p

Similarly, we can also treat the ranks provided by RRA as a probability distribution to which we can filter interactions
according to how robustly and highly ranked they are across the different methods.

In [ ]:
my_p = li.pl.dotplot(adata, 
                     colour='lrscore',
                     size='specificity_rank',
                     inverse_size=True,
                     source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
                     target_labels=['CD34+', 'CD56+ NK'],
                     filterby='steady_rank',
                     filter_lambda=lambda x: x <= 0.01,
                    )

In [ ]:
my_p

Finally, the plots in liana are built with `plotnine` and their aesthetics can be easily modified. For example:

In [ ]:
# we import plotnine
import plotnine as p9

In [ ]:
(my_p +
 # change theme
 p9.theme_dark() +
 # modify theme
 p9.theme(
     # adjust facet size
     strip_text=p9.element_text(size=12), 
     # rotate x axis text
     axis_text_x =p9.element_text(angle = 90)
 )
)

For more plot modification options  we refer the user to `plotnine`'s tutorials
and to the following link for a quick intro: 
https://datacarpentry.org/python-ecology-lesson/07-visualization-ggplot-python/index.html.